In [1]:
import graphviz
import pywinauto
import os
import subprocess
from PyQt5.QtWidgets import QApplication, QFileDialog
from PyQt5.QtCore import Qt
import re
import time

In [2]:
def get_species_names(dot_file):
    #read the dot file
    dot_file = graphviz.Source.from_file(dot_file)
    #make regex to find the labels, returning only the species names. The regex looks for the letters and numbers between label=" and ", while ignoring values that don't have a letter in them
    regex = r'(?<=label=")[A-Za-z0-9]+(?=")'

    #use regex to get labels from the source
    labels = dot_file.source
    #get the labels from the source using regex
    return(re.findall(regex, labels))

def displayed_species(pplot_list):
    return [item.texts()[0] for item in pplot_list.children()]

In [3]:
%gui qt

In [4]:
with QApplication([]) as app:
    file_dlg = QFileDialog()
    file_dlg.setFileMode(QFileDialog.ExistingFiles)
    #set initial directory
    file_dlg.setDirectory('C:\\Users\\KWILKES\\Desktop\\PFAS_Modeling\\cfs_runs\\0000_docs')
    file_dlg.setWindowFlags(file_dlg.windowFlags() | Qt.WindowStaysOnTopHint)
    file_dlg.raise_()
    file_dlg.activateWindow()
    file_dlg.setAcceptMode(QFileDialog.AcceptSave)
    file_names, _ = file_dlg.getOpenFileNames(None, "Select .netj file", "", "netj Files (*.netj)", options=QFileDialog.DontUseNativeDialog)
    if not file_names:
        app.quit()
    #close the dialog box
    file_dlg.close()
    app.quit()

    netj_files = file_names

In [ ]:
os.chdir("C:\\Users\\KWILKES\\AppData\\Local\\Programs\\ReactionEngineeringInternational\\CFS-EPA\\cowtip\\x64\\Release")
#execute run.bat but don't wait for it to finish before moving on
subprocess.Popen("run.bat")


# create an Application object and connect to the window
print("trying to connect")
app = pywinauto.Application(backend='uia').connect(title="CFS-EPA", timeout=20, path='C:\\Users\\KWILKES\\AppData\\Local\\Programs\\ReactionEngineeringInternational\\CFS-EPA\\cowtip\\x64\\Release\\Qt_Ui.exe')
main_window = app.MainWindow
#remove the last part of the name that goes {one or two numbers}_stream, and remove the last set of numbers from the beginning
pattern = r"(\d{2}\.\d{2,3})\.\d+(_[^_]+_[^_]+_[^_]+)_\d+stream\.netj$"
replacement = r"\1\2/dotfile.dot"
# interact with the MainWindow object to control the CFS-EPA application
for netj_file in netj_files:
    os.chdir("C:\\Users\\KWILKES\\Desktop\\PFAS_Modeling")
    dir_name = os.path.dirname(netj_file)
    base_name = os.path.basename(netj_file)
    # Apply the regex substitution
    new_base_name = re.sub(pattern, replacement, base_name)

    # Join the modified filename with the original directory
    dot_file = os.path.join(os.getenv('USERPROFILE'),'Desktop','PFAS_Modeling','pseudo_PFR','Output', new_base_name).replace('\\','/')

    main_window.set_focus()

    menu_bar=main_window.child_window(auto_id = "Qt_UiClass.menuBar")

    file_button=menu_bar.child_window(title="File")
    #get the file name and remove the path from netj_file
    netj_file_name=os.path.basename(netj_file)
    file_button.click_input()

    file_menu=file_button.child_window(auto_id="Qt_UiClass.menuBar.menuFile")
    load_button=file_menu.child_window(title="Load")
    load_button.click_input()
    #switch to newly created window named "Load"
    load_window=app.Dialog
    load_window.set_focus()
    file_name_box=load_window.child_window(auto_id="1148", control_type="Edit")
    file_name_box.set_text(f'"{netj_file_name}"')
    load_window.child_window(auto_id='1',title="Open", control_type='Button').click_input()
    run_window=app.window(title='CFS_EPA - '+netj_file.replace('\\','/'))
    run_window.child_window(title = "Rainbow (0) - UI", control_type="Button").click_input()


    exec_window = run_window.child_window(title="Qt_Ui_Rainbow", control_type="Window")
    exec_window.child_window(title="Execute", control_type="Button").click_input()
    stoplight_window=app.window(title='Qt_Ui_Rainbow_b', control_type = "Window")
    stoplight_window.child_window(title="Sreamline Destruction", control_type="Button", auto_id="Qt_Ui_Rainbow_bClass.centralWidget.streamlineDestruction_pushButton").click_input()
    streamline_window = app.window(auto_id='Qt_Ui_Rainbow_Streamline_FClass', control_type = "Window")

    stream_list=streamline_window.child_window(auto_id="Qt_Ui_Rainbow_Streamline_FClass.centralWidget.tabWidget.qt_tabwidget_stackedwidget.tab.listWidget_streamlines", control_type="List")
    #get the centerpoint of the rectangle that contains the list of streamlines
    stream_list_center=stream_list.rectangle().mid_point()

    for i in range(1, 10):
        streamline_window.child_window(title=str(i)).click_input()

    pywinauto.mouse.scroll(coords=(stream_list_center.x,stream_list_center.y), wheel_dist=-10)

    for i in range(10,17):
        streamline_window.child_window(title=str(i)).click_input()

    streamline_window.child_window(title="PPLOTS", control_type="TabItem").click_input()

    pplot_list=streamline_window.child_window(auto_id="Qt_Ui_Rainbow_Streamline_FClass.centralWidget.tabWidget.qt_tabwidget_stackedwidget.tab_4.listWidget_pplot", control_type="List")
    pplot_list_center=pplot_list.rectangle().mid_point()


    os.chdir("C:\\Users\\KWILKES\\Desktop\\PFAS_Modeling")
    spec_list  = ['TEMPERATURE:', 'PRESSURE:','MEAN_MW:']+get_species_names(dot_file)
    selected_species=[]



    #use the scroll wheel to scroll through the list of species, selecting species that are in spec_list and not yet selected. If the end of the list is reached and not all species are selected, print what is missing and continue
    while len(selected_species) < len(spec_list):
        for item in pplot_list.children():
            if item.texts()[0] in spec_list and item.texts()[0] not in selected_species:
                item.click_input()
                selected_species.append(item.texts()[0])

        #if the displayed items don't change after scrolling, the end of the list has been reached        
        pre_scroll_items=displayed_species(pplot_list)
        pywinauto.mouse.scroll(coords=(pplot_list_center.x,pplot_list_center.y), wheel_dist=-3)
        post_scroll_items=displayed_species(pplot_list)
        if pre_scroll_items==post_scroll_items:
            print ("end of list reached")
            if len(selected_species) < len(spec_list):
                missed_species=[item for item in spec_list if item not in selected_species]
                print(f"missing species: {missed_species}")
                break
            break
    if len(selected_species) < len(spec_list):
        print("not all species selected")
    #click the "Plot" button
    streamline_window.child_window(title="generate pplots", control_type="Button").click_input()
    # print(app.windows())
    #close all windows except for the one that starts with CFS_EPA
    #wait for program to finish before closing windows
    pywinauto.timings.Timings.window_find_timeout = 10
    start_time = time.time()
    while True:
        try:
            streamline_window.set_focus()
            break
        except TimeoutError or pywinauto.findwindows.ElementNotFoundError:
            time.sleep(10)
            elapsed_time = time.time() - start_time
            print(f"elapsed time: {elapsed_time}")
            if elapsed_time > 1000:
                print("timed out")
                break
    #close windows that don't have "CFS_EPA" in the title and set the one with "CFS_EPA" in the title to the main window
    for window in app.windows():
        if "CFS_EPA" not in window.window_text():
            window.close()
        else:
            main_window=app.window(title='CFS_EPA - '+netj_file.replace('\\','/'))